In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.impute import SimpleImputer

# Wczytanie danych (korzystamy z wersji online dla ułatwienia, lub lokalnego pliku)
url = "https://raw.githubusercontent.com/joolsa/demo_real_estate/master/data/ames_housing_trimmed_processed.csv"
try:
    df = pd.read_csv(url)
    print("Dane wczytane pomyślnie z URL.")
except:
    # Jeśli masz plik lokalnie:
    df = pd.read_csv('AMES_housing_Price.csv')
    print("Dane wczytane z pliku lokalnego.")

# Sprawdzenie danych
print(f"Rozmiar zbioru: {df.shape}")
df.head()
# 1. Wybór tylko kolumn numerycznych
df_num = df.select_dtypes(include=[np.number]).copy()

# 2. Obsługa brakujących danych (imputacja medianą)
imputer = SimpleImputer(strategy='median')
df_num_imputed = pd.DataFrame(imputer.fit_transform(df_num), columns=df_num.columns)

# 3. Podział na X (cechy) i y (cel - SalePrice)
# Zakładamy, że kolumna docelowa nazywa się 'SalePrice'
X = df_num_imputed.drop('SalePrice', axis=1)
y = df_num_imputed['SalePrice']

# 4. Podział na zbiór treningowy i testowy (str. 35 prezentacji)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 5. Budowa modelu regresji liniowej
model = LinearRegression()
model.fit(X_train, y_train)

# 6. Predykcja i ocena (RMSE i R2 - str. 17 i 21)
y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"Model Bazowy (tylko liczby):")
print(f"RMSE: {rmse:.2f}")
print(f"R2 Score: {r2:.4f}")
# Sprawdzenie korelacji zmiennych z ceną (SalePrice)
korelacje = df_num_imputed.corr()['SalePrice'].sort_values(ascending=False)

print("Top 10 zmiennych najsilniej skorelowanych z ceną:")
print(korelacje.head(10))

# Wybór zmiennych o korelacji > 0.5 (silny związek)
wybrane_cechy = korelacje[korelacje > 0.5].index.tolist()
wybrane_cechy.remove('SalePrice') # Usuwamy cel z listy cech

print(f"\nWybrane cechy do lepszego modelu: {wybrane_cechy}")

# Trenowanie modelu na wybranych cechach
X_sel = df_num_imputed[wybrane_cechy]
X_train_sel, X_test_sel, y_train_sel, y_test_sel = train_test_split(X_sel, y, test_size=0.3, random_state=42)

model_sel = LinearRegression()
model_sel.fit(X_train_sel, y_train_sel)
y_pred_sel = model_sel.predict(X_test_sel)

rmse_sel = np.sqrt(mean_squared_error(y_test_sel, y_pred_sel))
print(f"\nModel po selekcji zmiennych - RMSE: {rmse_sel:.2f}")
# 1. Pobranie zmiennych kategorycznych (tekstowych)
df_cat = df.select_dtypes(include=['object'])

# 2. One-Hot Encoding z usuwaniem pierwszej kolumny (Reference Coding - str. 56-61)
# drop_first=True jest kluczowe dla regresji liniowej!
df_cat_encoded = pd.get_dummies(df_cat, drop_first=True)

# 3. Łączenie wybranych cech numerycznych i zakodowanych kategorycznych
X_final = pd.concat([df_num_imputed[wybrane_cechy], df_cat_encoded], axis=1)

# Uzupełnienie ewentualnych braków w nowych kolumnach zerami
X_final = X_final.fillna(0)

print(f"Nowy wymiar danych wejściowych: {X_final.shape}")
# Używamy walidacji krzyżowej (k-Fold, k=5) do oceny ostatecznego modelu
model_final = LinearRegression()

# cross_val_score zwraca "neg_root_mean_squared_error", więc musimy wziąć wartość przeciwną
cv_scores = cross_val_score(model_final, X_final, y, scoring='neg_root_mean_squared_error', cv=5)

# Zamiana na wartości dodatnie RMSE
rmse_scores = -cv_scores

print(f"Wyniki RMSE dla 5 foldów: {rmse_scores}")
print(f"Średnie RMSE (Walidacja krzyżowa): {rmse_scores.mean():.2f}")
print(f"Odchylenie standardowe: {rmse_scores.std():.2f}")

Dane wczytane z pliku lokalnego.
Rozmiar zbioru: (1460, 81)
Model Bazowy (tylko liczby):
RMSE: 35273.66
R2 Score: 0.8217
Top 10 zmiennych najsilniej skorelowanych z ceną:
SalePrice       1.000000
OverallQual     0.790982
GrLivArea       0.708624
GarageCars      0.640409
GarageArea      0.623431
TotalBsmtSF     0.613581
1stFlrSF        0.605852
FullBath        0.560664
TotRmsAbvGrd    0.533723
YearBuilt       0.522897
Name: SalePrice, dtype: float64

Wybrane cechy do lepszego modelu: ['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF', '1stFlrSF', 'FullBath', 'TotRmsAbvGrd', 'YearBuilt', 'YearRemodAdd']

Model po selekcji zmiennych - RMSE: 37253.02
Nowy wymiar danych wejściowych: (1460, 218)
Wyniki RMSE dla 5 foldów: [59096.49095825 36677.08894204 39870.68505806 25438.01494722
 43897.69157325]
Średnie RMSE (Walidacja krzyżowa): 40995.99
Odchylenie standardowe: 10936.34
